In [23]:
import xgt
import os
import pandas

from platform import python_version
print (python_version())

3.7.4


In [24]:
if os.environ.get('https_proxy'):
 del os.environ['https_proxy']
if os.environ.get('http_proxy'):
 del os.environ['http_proxy']

In [25]:
conn=xgt.Connection()
conn.server_version

'1.3.0'

In [26]:
try:
  devices = conn.get_vertex_frame('Devices')
except xgt.XgtNameError:
  devices = conn.create_vertex_frame(
      name='Devices',
      schema=[['device', xgt.TEXT]],
      key='device')
devices

In [27]:
try:
  netflow = conn.get_edge_frame('Netflow')
except xgt.XgtNameError:
  netflow = conn.create_edge_frame(
      name='Netflow',
      schema=[['epoch_time', xgt.INT],
              ['duration', xgt.INT],
              ['src_device', xgt.TEXT],
              ['dst_device', xgt.TEXT],
              ['protocol', xgt.INT],
              ['src_port', xgt.INT],
              ['dst_port', xgt.INT],
              ['src_packets', xgt.INT],
              ['dst_packets', xgt.INT],
              ['src_bytes', xgt.INT],
              ['dst_bytes', xgt.INT]],
      source=devices,
      target=devices,
      source_key='src_device',
      target_key='dst_device')
netflow

In [28]:
try:
  host_events = conn.get_edge_frame('HostEvents')
except xgt.XgtNameError:
  host_events = conn.create_edge_frame(
      name='HostEvents',
      schema=[['epoch_time', xgt.INT],
              ['event_id', xgt.INT],
              ['log_host', xgt.TEXT],
              ['user_name', xgt.TEXT],
              ['domain_name', xgt.TEXT],
              ['logon_id', xgt.INT],
              ['process_name', xgt.TEXT],
              ['process_id', xgt.INT],
              ['parent_process_name', xgt.TEXT],
              ['parent_process_id', xgt.INT]],
           source=devices,
           target=devices,
           source_key='log_host',
           target_key='log_host')
host_events

In [29]:
try:
  auth_events = conn.get_edge_frame('AuthEvents')
except xgt.XgtNameError:
  auth_events = conn.create_edge_frame(
           name='AuthEvents',
           schema = [['epoch_time',xgt.INT],
                     ['event_id',xgt.INT],
                     ['log_host',xgt.TEXT],
                     ['logon_type',xgt.INT],
                     ['logon_type_description',xgt.TEXT],
                     ['user_name',xgt.TEXT],
                     ['domain_name',xgt.TEXT],
                     ['logon_id',xgt.INT],
                     ['subject_user_name',xgt.TEXT],
                     ['subject_domain_name',xgt.TEXT],
                     ['subject_logon_id',xgt.TEXT],
                     ['status',xgt.TEXT],
                     ['src',xgt.TEXT],
                     ['service_name',xgt.TEXT],
                     ['destination',xgt.TEXT],
                     ['authentication_package',xgt.TEXT],
                     ['failure_reason',xgt.TEXT],
                     ['process_name',xgt.TEXT],
                     ['process_id',xgt.INT],
                     ['parent_process_name',xgt.TEXT],
                     ['parent_process_id',xgt.INT]],
            source = 'Devices',
            target = 'Devices',
            source_key = 'src',
            target_key = 'destination')
auth_events

In [30]:
# Utility to print the sizes of data currently in xGT
def print_data_summary():
  print('Devices (vertices): {:,}'.format(devices.num_vertices))
  print('Netflow (edges): {:,}'.format(netflow.num_edges))
  print('Host events (edges): {:,}'.format(host_events.num_edges))
  print('Authentication events (edges): {:,}'.format(auth_events.num_edges))
  print('Total (edges): {:,}'.format(
      netflow.num_edges + host_events.num_edges + auth_events.num_edges))
    
print_data_summary()

Devices (vertices): 0
Netflow (edges): 0
Host events (edges): 0
Authentication events (edges): 0
Total (edges): 0


In [31]:
%%time

# Load the HostEvents event data:
if host_events.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/wls_day-85_1v.csv"]
    # urls = ["xgtd://nvme_data1/data_1v/wls_day-{:02d}_1v.csv".format(_) for _ in range(2,91)]
    # urls = ["xgtd://nvme_data1/data_1v/wls_day-11_1v.csv"]
    urls = ["xgtd://nvme_data1/data_1v/wls_day-85_1v.csv"]
    host_events.load(urls)
    print_data_summary()

Devices (vertices): 10,324
Netflow (edges): 0
Host events (edges): 18,637,483
Authentication events (edges): 0
Total (edges): 18,637,483
CPU times: user 14.9 ms, sys: 11 ms, total: 25.9 ms
Wall time: 23.3 s


In [32]:
%%time

# Load the AuthEvents event data:
if auth_events.num_edges == 0:
    # urls = ["https://datasets.trovares.com/LANL/xgt/wls_day-85_2v.csv"]
    # urls = ["xgtd://nvme_data9/data_2v/output/wls_day-{:02d}_2v.csv".format(_) for _ in range(2,91)]
    # urls = ["xgtd://nvme_data9/data_2v/output/wls_day-11_2v.csv"]
    # urls = ["xgtd://nvme_data9/data_2v/output/wls_day-85_2v.csv"]
    urls = ["xgtd://nvme_data3/data_2v/wls_day-85_2v.csv"]
    auth_events.load(urls)
    print_data_summary()

Devices (vertices): 12,288
Netflow (edges): 0
Host events (edges): 18,637,483
Authentication events (edges): 47,790,045
Total (edges): 66,427,528
CPU times: user 36.2 ms, sys: 43.3 ms, total: 79.5 ms
Wall time: 1min 16s


In [33]:
%%time

# Load the netflow data:
if netflow.num_edges == 0:
    #urls = ["https://datasets.trovares.com/LANL/xgt/nf_day-85.csv"]
    #urls = ["xgtd://nvme_data1/data_nf/nf_day-{:02d}.csv".format(_) for _ in range(2,91)]
    # urls = ["xgtd://nvme_data1/data_nf/nf_day-11.csv"]
    # urls = ["xgtd://nvme_data1/data_nf/nf_day-85.csv"]
    urls = ["xgtd://nvme_data5/data_nf/nf_day-85.csv"]
    netflow.load(urls)
    print_data_summary()

Devices (vertices): 137,812
Netflow (edges): 235,661,328
Host events (edges): 18,637,483
Authentication events (edges): 47,790,045
Total (edges): 302,088,856
CPU times: user 80.9 ms, sys: 45.3 ms, total: 126 ms
Wall time: 2min 59s


In [34]:
# Utility function to launch queries and show job number:
#   The job number may be useful if a long-running job needs
#   to be canceled.

def run_query(query, table_name = "answers", drop_answer_table=True, show_query=False):
    if drop_answer_table:
        conn.drop_frame(table_name)
    if query[-1] != '\n':
        query += '\n'
    query += 'INTO {}'.format(table_name)
    if show_query:
        print("Query:\n" + query)
    job = conn.schedule_job(query)
    print("Launched job {}".format(job.id))
    conn.wait_for_job(job)
    table = conn.get_table_frame(table_name)
    return table

In [35]:
# Generate a new edge frame for holding only the RDP edges
import time
query_start_time = time.time()

conn.drop_frame('RDPFlow')
rdp_flow = conn.create_edge_frame(
            name='RDPFlow',
            schema=netflow.schema,
            source=devices,
            target=devices,
            source_key='src_device',
            target_key='dst_device')
rdp_flow

In [36]:
%%time

#Extract forward RDP edges
q = """
MATCH (v0)-[edge:Netflow]->(v1)
WHERE edge.dst_port=3389
CREATE (v0)-[e:RDPFlow {epoch_time : edge.epoch_time,
  duration : edge.duration, protocol : edge.protocol,
  src_port : edge.src_port, dst_port : edge.dst_port,
  src_packets : edge.src_packets, dst_packets : edge.dst_packets,
  src_bytes : edge.src_bytes, dst_bytes : edge.dst_bytes}]->(v1)
RETURN count(*)
"""
data = run_query(q)
print('Number of answers: {:,}'.format(data.get_data()[0][0]))

Launched job 2686
Number of answers: 31
CPU times: user 10.1 ms, sys: 183 µs, total: 10.3 ms
Wall time: 1.01 s


In [37]:
%%time

#Extract reverse RDP edges
q = """
MATCH (v0)-[edge:Netflow]->(v1)
WHERE edge.src_port=3389
CREATE (v1)-[e:RDPFlow {epoch_time : edge.epoch_time,
  duration : edge.duration, protocol : edge.protocol,
  src_port : edge.dst_port, dst_port : edge.src_port,
  src_packets : edge.dst_packets, dst_packets : edge.src_packets,
  src_bytes : edge.dst_bytes, dst_bytes : edge.src_bytes}]->(v0)
RETURN count(*)
"""
data = run_query(q)
print('Number of answers: {:,}'.format(data.get_data()[0][0]))

Launched job 2724
Number of answers: 9,701
CPU times: user 5.4 ms, sys: 4.75 ms, total: 10.1 ms
Wall time: 1.02 s


In [38]:
data=None
if rdp_flow.num_edges == 0:
    print("RDPFlow is empty")
elif rdp_flow.num_edges <= 1000:
    data = rdp_flow.get_data_pandas()
else:
    data = 'RDPflow (edges): {:,}'.format(rdp_flow.num_edges)
data

'RDPflow (edges): 9,732'

In [39]:
# Utility to print the data sizes currently in xGT
def print_netflow_data_summary():
  print_data_summary()
  print('RDPFlow (edges): {:,}'.format(rdp_flow.num_edges))

print_netflow_data_summary()

Devices (vertices): 137,812
Netflow (edges): 235,661,328
Host events (edges): 18,637,483
Authentication events (edges): 47,790,045
Total (edges): 302,088,856
RDPFlow (edges): 9,732


In [40]:
%%time

#Lateral Movement Query
time_threshold_between_step = 3600   # one hour
time_threshold_hijack = 180          # three minutes
time_threshold_one_step = 480        # eight minutes
q = """
MATCH (A)-[rdp1:RDPFlow]->(B)-[rdp2:RDPFlow]->(C),
      (A)-[hijack1:HostEvents]->(A)-[privEsc1:HostEvents]->(A),
      (B)-[hijack2:HostEvents]->(B)-[privEsc2:HostEvents]->(B)
WHERE A <> B AND B <> C AND A <> C 
  AND privEsc1.event_id = 4688 
  AND (privEsc1.process_name = "Proc336322.exe" OR privEsc1.process_name = "Proc695356.exe")
  AND hijack1.event_id = 4688 AND hijack1.process_name = "Proc249569.exe"
  AND privEsc2.event_id = 4688 
  AND (privEsc2.process_name = "Proc336322.exe" OR privEsc2.process_name = "Proc695356.exe")
  AND hijack2.event_id = 4688 AND hijack2.process_name = "Proc249569.exe"

  // Check time constraints on the overall pattern
  AND rdp1.epoch_time <= rdp2.epoch_time
  AND rdp2.epoch_time - rdp1.epoch_time < {0}

  // Check time constraints on step from A to B
  AND privEsc1.epoch_time <= hijack1.epoch_time
  AND hijack1.epoch_time <= rdp1.epoch_time
  AND rdp1.epoch_time - hijack1.epoch_time < {1}
  AND rdp1.epoch_time - privEsc1.epoch_time < {2}

  // Check time constraints on step from B to C
  AND privEsc2.epoch_time <= hijack2.epoch_time
  AND hijack2.epoch_time <= rdp2.epoch_time
  AND rdp2.epoch_time - hijack2.epoch_time < {1}
  AND rdp2.epoch_time - privEsc2.epoch_time < {2}
RETURN rdp1.src_device, rdp1.dst_device, rdp1.epoch_time, rdp2.dst_device, rdp2.epoch_time
""".format(time_threshold_between_step, time_threshold_hijack, time_threshold_one_step)
answer_table = run_query(q)
print('Number of answers: {:,}'.format(answer_table.num_rows))

Launched job 2746
Number of answers: 10,572
CPU times: user 6.24 ms, sys: 1.3 ms, total: 7.54 ms
Wall time: 1.42 s


In [41]:
%%time
# Build HijackEvents table

import time
start_optimized_query_time = time.time()

conn.drop_frame('HijackEvents')
hijack_events = conn.create_edge_frame(
    name   ='HijackEvents',
    schema = [['epoch_time', xgt.INT],
              ['src_host', xgt.TEXT],
              ['dst_host', xgt.TEXT]],
    source = devices,
    target = devices,
    source_key = 'src_host',
    target_key = 'dst_host')

query = """
MATCH (v0)-[edge:HostEvents]->(v0)
WHERE edge.process_name = "Proc249569.exe"
  AND edge.event_id = 4688
CREATE (v0)-[e:HijackEvents { epoch_time : edge.epoch_time }]->(v0)
RETURN count(*)
"""
run_query(query)
print('HijackEvents (edges): {:,}'.format(hijack_events.num_edges))

Launched job 2752
HijackEvents (edges): 166,889
CPU times: user 4.79 ms, sys: 1.71 ms, total: 6.49 ms
Wall time: 335 ms


In [42]:
%%time
# Build a PrivEscEvents table

conn.drop_frame('PrivEscEvents')
priv_esc_events = conn.create_edge_frame(
    name   ='PrivEscEvents',
    schema = [['epoch_time', xgt.INT],
              ['src_host', xgt.TEXT],
              ['dst_host', xgt.TEXT]],
    source = devices,
    target = devices,
    source_key = 'src_host',
    target_key = 'dst_host')

query = """
MATCH (v0)-[edge:HostEvents]->(v0)
WHERE edge.process_name = "Proc336322.exe" OR
      edge.process_name = "Proc695356.exe"
  AND edge.event_id = 4688
CREATE (v0)-[e:PrivEscEvents { epoch_time : edge.epoch_time }]->(v0)
RETURN count(*)
"""
run_query(query)
print('PrivEscEvents (edges): {:,}'.format(priv_esc_events.num_edges))

Launched job 2780
PrivEscEvents (edges): 135,214
CPU times: user 4.46 ms, sys: 1.03 ms, total: 5.5 ms
Wall time: 382 ms


In [43]:
%%time
# Now run the lateral movement query using these new index tables

q = """
MATCH (A)-[rdp1:RDPFlow]->(B)-[rdp2:RDPFlow]->(C),
      (A)-[hijack1:HijackEvents]->(A)-[priv_esc1:PrivEscEvents]->(A),
      (B)-[hijack2:HijackEvents]->(B)-[priv_esc2:PrivEscEvents]->(B)
WHERE A <> B AND B <> C AND A <> C 
  // Check time constraints on the overall pattern
  AND rdp1.epoch_time <= rdp2.epoch_time
  AND rdp2.epoch_time - rdp1.epoch_time < {0}

  // Check time constraints on step from A to B
  AND priv_esc1.epoch_time <= hijack1.epoch_time
  AND hijack1.epoch_time <= rdp1.epoch_time
  AND rdp1.epoch_time - hijack1.epoch_time < {1}
  AND rdp1.epoch_time - priv_esc1.epoch_time < {2}

  // Check time constraints on step from B to C
  AND priv_esc2.epoch_time <= hijack2.epoch_time
  AND hijack2.epoch_time <= rdp2.epoch_time
  AND rdp2.epoch_time - hijack2.epoch_time < {1}
  AND rdp2.epoch_time - priv_esc2.epoch_time < {2}
RETURN rdp1.src_device, rdp1.dst_device, rdp1.epoch_time, rdp2.dst_device, rdp2.epoch_time
""".format(time_threshold_between_step, time_threshold_hijack, time_threshold_one_step)
start_timer = time.time()
answer_table = run_query(q)
end_timer = time.time()
query_end_time = time.time()
print('Number of answers: {:,}'.format(answer_table.num_rows))

print("Time for query: {:,.2f}".format(end_timer - start_timer))
print("Overall time for query: {:,.2f}".format(query_end_time - query_start_time))

Launched job 2818
Number of answers: 10,572
Time for query: 0.91
Overall time for query: 5.16
CPU times: user 4.36 ms, sys: 0 ns, total: 4.36 ms
Wall time: 907 ms


In [44]:
# retrieve the answer rows to the client in a pandas frame
data = answer_table.get_data_pandas()
data[0:10]

,rdp1_src_device,rdp1_dst_device,rdp1_epoch_time,rdp2_dst_device,rdp2_epoch_time
0,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
1,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
2,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
3,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
4,ActiveDirectory,EnterpriseAppServer,7290438,Comp503532,7291140
5,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
6,ActiveDirectory,EnterpriseAppServer,7290438,Comp503532,7291140
7,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
8,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
9,ActiveDirectory,EnterpriseAppServer,7290427,Comp503532,7291140
